<h1>Read & Summarize Trajactory Files.ipynb</h1>

In [1]:
import dask.dataframe as dd
import pandas as pd

In [2]:
df = dd.read_parquet('../data/trajactory_files/*.parquet')
#df = dd.read_parquet('../data/trajactory_files/2022-01-01.parquet')

unique_values = df['flight_id'].unique().compute()
print(f"{len(unique_values) = }\n")

display(df.head())

display(df.tail())

len(unique_values) = 26231



,flight_id,timestamp,latitude,longitude,altitude,groundspeed,track,vertical_rate,track_unwrapped,u_component_of_wind,v_component_of_wind,temperature,specific_humidity,icao24
0,248763780,2022-01-01 14:04:41+00:00,51.477651,-0.443802,25.0,157.0,271.859599,1216.0,271.859599,NaN,NaN,NaN,NaN,248763780
1,248763780,2022-01-01 14:04:42+00:00,51.477651,-0.443802,25.0,157.0,271.859599,1216.0,271.859599,NaN,NaN,NaN,NaN,248763780
2,248763780,2022-01-01 14:04:43+00:00,51.477651,-0.443802,25.0,157.0,271.859599,1216.0,271.859599,NaN,NaN,NaN,NaN,248763780
3,248763780,2022-01-01 14:04:44+00:00,51.477651,-0.443802,25.0,157.0,271.859599,1216.0,271.859599,NaN,NaN,NaN,NaN,248763780
4,248763780,2022-01-01 14:04:45+00:00,51.477651,-0.443802,25.0,157.0,271.859599,1216.0,271.859599,NaN,NaN,NaN,NaN,248763780


,flight_id,timestamp,latitude,longitude,altitude,groundspeed,track,vertical_rate,track_unwrapped,u_component_of_wind,v_component_of_wind,temperature,specific_humidity,icao24
5662697,249226222,2022-01-28 12:24:26+00:00,51.464951,-0.438934,-525.0,111.0,269.483836,-64.0,269.483836,NaN,NaN,NaN,NaN,249226222
5662698,249226222,2022-01-28 12:24:27+00:00,51.464951,-0.438934,-525.0,111.0,269.483836,-64.0,269.483836,NaN,NaN,NaN,NaN,249226222
5662699,249226222,2022-01-28 12:24:28+00:00,51.464951,-0.438934,-525.0,111.0,269.483836,-64.0,269.483836,NaN,NaN,NaN,NaN,249226222
5662700,249226222,2022-01-28 12:24:29+00:00,51.464951,-0.438934,-525.0,111.0,269.483836,-64.0,269.483836,NaN,NaN,NaN,NaN,249226222
5662701,249226222,2022-01-28 12:24:30+00:00,51.464951,-0.438934,-525.0,111.0,269.483836,-64.0,269.483836,NaN,NaN,NaN,NaN,249226222


In [3]:
def calculate_stat(
    dataframe: pd.DataFrame, 
    group_by_column: str, 
    target_column: str,
    stat_type: str,
    value_type: str = 'float64'
) -> pd.Series | None:
    result = None
    if stat_type == 'median':
        result = df.groupby(group_by_column).apply(lambda group: group[target_column].median(), meta=('value', value_type)).compute()
    elif stat_type == 'mean':
        result = df.groupby(group_by_column).apply(lambda group: group[target_column].mean(), meta=('value', value_type)).compute()
    return result

In [ ]:
latitude_median = calculate_stat(dataframe=df, group_by_column='flight_id', target_column='latitude', stat_type='median')
longitude_median = calculate_stat(dataframe=df, group_by_column='flight_id', target_column='longitude', stat_type='median')
altitude_mean = calculate_stat(dataframe=df, group_by_column='flight_id', target_column='altitude', stat_type='mean', value_type='int64')
groundspeed_mean = calculate_stat(dataframe=df, group_by_column='flight_id', target_column='groundspeed', stat_type='mean', value_type='int64')
track_mean = calculate_stat(dataframe=df, group_by_column='flight_id', target_column='track', stat_type='mean')
vertical_rate_mean = calculate_stat(dataframe=df, group_by_column='flight_id', target_column='vertical_rate', stat_type='mean', value_type='int64')
track_unwrapped_mean = calculate_stat(dataframe=df, group_by_column='flight_id', target_column='track_unwrapped', stat_type='mean')
u_component_of_wind_mean = calculate_stat(dataframe=df, group_by_column='flight_id', target_column='u_component_of_wind', stat_type='mean')
v_component_of_wind_mean = calculate_stat(dataframe=df, group_by_column='flight_id', target_column='v_component_of_wind', stat_type='mean')
temperature_mean = calculate_stat(dataframe=df, group_by_column='flight_id', target_column='temperature', stat_type='mean')
specific_humidity_mean = calculate_stat(dataframe=df, group_by_column='flight_id', target_column='specific_humidity', stat_type='mean')

trajactory_summary_df = pd.DataFrame({
    "latitude_median": latitude_median,
    "longitude_median": longitude_median,
    "altitude_mean": altitude_mean,
    "groundspeed_mean": groundspeed_mean,
    "track_mean": track_mean,
    "vertical_rate_mean": vertical_rate_mean,
    "track_unwrapped_mean": track_unwrapped_mean,
    "u_component_of_wind_mean": u_component_of_wind_mean,
    "v_component_of_wind_mean": v_component_of_wind_mean,
    "temperature": temperature_mean,
    "specific_humidity": specific_humidity_mean
})

display(trajactory_summary_df)

In [ ]:
def save_dataframe(path: str, dataframe: pd.DataFrame) -> None:
    dataframe.to_csv(path, index=False)
    print(f"{path} is saved!")

In [ ]:
save_dataframe(path="../data/trajactory_summary.csv", dataframe=trajactory_summary_df)